# LangSmith API Key 로드

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

# Langsmith로 프로젝트 추적 설정

In [2]:
from utils import logging

logging.langsmith("Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
Runnable


# Runnable

사용자 정의 체인을 쉽게 만들 수 있는 프로토콜이며, 데이터를 효과적으로 전달할 수 있습니다.

## 기존 방식

In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0)
chain = prompt | llm

chain.invoke({"num": 5}) # 기본
chain.invoke(5) # 1개의 변수만 있을 때 이렇게도 사용 가능

AIMessage(content='50입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7ca5ab83-69ac-44c5-a1ac-0a4417208751-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## Runnable 객체를 사용한 방식

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0)

runnable_chain = {"num": RunnablePassthrough()} | prompt | llm
runnable_chain.invoke(10)

기존 방식은 딕셔너리 형태로 입력값을 전달해야 하며, 단순하게 특정 변수에 값을 바로 치환하는 방식입니다.

Runnable 객체를 이용한 방식은 입력값을 가공하거나 유연하게 처리할 수 있는 구조를 제공합니다. 

# RunnablePassthrough

RunnablePassthrough 는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다.

- RunnablePassthrough() 가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.

- RunnablePassthrough.assign(...) 방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

## RunnablePassthrough()

In [33]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [34]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | llm
runnable_chain.invoke(10)

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-650d0135-d18d-45a1-807e-043ec79d977c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## RunnablePassthrough.assign()

In [55]:
# 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합칩니다.
# 입력 키 : num, 할당(assign) 키 : new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1}) 


{'num': 1, 'new_num': 3}

# RunnableParallel

RunnableParallel 인스턴스를 생성합니다.

이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.

In [56]:
from langchain_core.runnables import RunnableParallel

# Runnableparallel 인스턴스 생성
runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    extra = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified = lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {"num": 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

Chain 도 RunnableParallel 적용할 수 있습니다.

In [ ]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

# RunnableLambda

RunnableLambda 는 사용자 정의 함수를 실행 할 수 있는 기능을 제공합니다.

이를 통해 개발자는 자신만의 함수를 정의하고, 해당 함수를 RunnableLambda 를 사용하여 실행할 수 있습니다.

예를 들어, 데이터 전처리, 계산, 또는 외부 API와의 상호 작용과 같은 작업을 수행하는 함수를 정의하고 실행할 수 있습니다.


#### 주의사항

사용자 정의 함수를 RunnableLambda로 래핑하여 활용할 수 있는데, 여기서 주의할 점은 사용자 정의 함수가 받을 수 있는 인자는 1개 뿐이라는 점 입니다.

만약, 여러 인수를 받는 함수로 구현하고 싶다면, 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼를 작성해야 합니다.

wrapper란?
- 래퍼(wrapper) 함수란, 다른 함수를 감싸서 호출하거나 그 함수의 동작을 확장, 변형, 또는 단순화하는 역할을 하는 함수입니다. 
래퍼 함수는 기존의 함수 호출 전에 추가적인 작업을 수행하거나, 함수의 결과를 처리하는 작업을 추가하고자 할 때 유용하게 사용됩니다.

In [3]:
from operator import itemgetter 
# operator.itemgetter는 주로 sorted와 같은 함수의 key 매개변수에 적용하여 다양한 기준으로 정렬할 수 있도록 하는 모듈

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


def length_function(text):  # 텍스트의 길이를 반환하는 함수
    return len(text)


def _multiple_length_function(text1, text2):  # 두 텍스트의 길이를 곱하는 함수
    """이름 앞에 _가 붙어 있으므로 내부용(private) 함수로 설계된 것"""
    return len(text1) * len(text2)


def multiple_length_function(  # 2개의 딕셔너리 인자를 받는 함수로 연결하는 wrapper 함수
    _dict,
):  # 딕셔너리에서 "text1"과 "text2"의 길이를 곱하는 함수
    """ 
    하나의 딕셔너리 _dict를 인자로 받고, 
    그 딕셔너리 내에서 text1과 text2라는 두 개의 키에 해당하는 값들을 추출해
    _multiple_length_function 함수에 전달
    """
    return _multiple_length_function(_dict["text1"], _dict["text2"])


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("what is {a} + {b}?")
# ChatOpenAI 모델 초기화
model = ChatOpenAI()

# 프롬프트와 모델을 연결하여 체인 생성
chain1 = prompt | model

# 체인 구성
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function), # 3
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")} | RunnableLambda(multiple_length_function), # 9
    }
    | prompt
    | model
    | StrOutputParser()
)

In [4]:
# 주어진 인자들로 체인을 실행합니다.
chain.invoke({"input_1": "bar", "input_2": "gah"})

'3 + 9 = 12'

## @chain 데코레이터를 사용해 Runnable 구성

@chain 데코레이터를 추가하여 임의의 함수를 체인으로 변환할 수 있습니다.

이는 함수를 RunnableLambda로 래핑하는 것과 기능적으로 동일합니다.

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

ChatPromptTemplate 클래스를 사용하여 두 개의 프롬프트 템플릿을 정의합니다.

In [6]:
# 프롬프트 템플릿을 정의합니다.
prompt1 = ChatPromptTemplate.from_template("{topic} 에 대해 짧게 한글로 설명해주세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{sentence} 를 emoji를 활용한 인스타그램 게시글로 만들어주세요."
)

custom_chain 함수는 입력 텍스트를 기반으로 사용자 정의 체인을 실행합니다.

@chain 데코레이터로 사용자 정의 함수를 데코레이팅 하며, 데코레이팅을 통해 함수를 Runnable 한 객체로 만듭니다.

In [7]:
@chain
def custom_chain(text):
    # 첫 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    chain1 = prompt1 | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
    output1 = chain1.invoke({"topic": text})

    # 두 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    chain2 = prompt2 | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
    # 두 번째 체인을 호출하여 파싱된 첫 번째 결과를 전달하고 최종 결과를 반환합니다.
    return chain2.invoke({"sentence": output1})

custom_chain은 이제 실행 가능한 객체(runnable)이므로, invoke() 를 사용하여 실행해야 합니다.

LangSmith 추적 기록을 확인해 보면, custom_chain 추적 정보가 있을 것이며, 그 아래에는 OpenAI 호출이 중첩되어 있을 것입니다.

In [8]:
# custom_chain을 호출
print(custom_chain.invoke("양자역학"))

🌌✨ 양자역학의 매력에 빠져보세요! 🔍💫

양자역학은 물리학의 한 분야로, 원자와 아원자 입자의 신비한 행동을 설명합니다. 🧬💥 고전 물리학과는 달리, 양자역학은 입자가 특정한 위치에 동시에 존재할 수 없고, 확률적인 성질을 가진다고 해요! 🎲🔮

입자의 상태는 관측하기 전까지 확정되지 않으며, 이는 '파동-입자 이중성'과 '불확정성 원리'로 유명하답니다. 🌊➡️⚛️✨ 현대 물리학의 기초를 이루며, 반도체, 레이저, 양자 컴퓨팅 등 다양한 기술의 발전에 기여하고 있어요! 💻⚡️

양자세계의 신비로움을 함께 탐험해봐요! 🌟🔭 #양자역학 #물리학 #과학의세계 #혁신 #기술발전 #파동입자이중성 #불확정성원리
